<a href="https://colab.research.google.com/github/textspur/prompting_open_llm/blob/main/Example20231211_tweets_sentiment_neuralChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

The script starts by downloading the necessary libraries.

In [ ]:
### Installation on Colab
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install wget

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=6db9502d80367f4cf9aa2d7983c031d3166bda3a4d5fcd851f2b72db39064ed4
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


# Access Google Drive to save results

In [ ]:
import os
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
output_dir = "/content/drive/My Drive/LLM_Prompting/Tweets/LLM_Prompts_Tweets20231211/results/neural-chat-7b-v3-2"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load Data

In [ ]:
import pandas as pd
import requests

# Define the URL and fetch the data
url = "https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_A/gold/SemEval2017-task4-test.subtask-A.english.txt"
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Split the response text into lines and then split each line by tab to get the data
data = [line.split('\t') for line in response.text.strip().split('\n')]

# Load data into pandas DataFrame
df = pd.DataFrame(data, columns=['id', 'sentiment', 'text'])


# Checking the first few rows of the DataFrame
print(df.head())
print("Length of dataframe df: ", len(df))

# delete empty rows if any
print("Length of dataframe df: ", len(df))
df = df.dropna(subset=['sentiment'])
print("Length of dataframe df: ", len(df))

# delete duplicates on id and text if any
df = df.drop_duplicates(subset='id', keep=False)
df = df.drop_duplicates(subset='text', keep=False)

print("Length of dataframe df: ", len(df))
print("Counts of values of sentiment column in dataframe df: \n", df['sentiment'].value_counts())

value_counts_percent = df['sentiment'].value_counts(normalize=True) * 100
print("Counts of values of sentiment column in dataframe df: \n", value_counts_percent)

print(len(df))
#df_random_sample10 = df.sample(n=10, random_state=1)

df_random_sample500 = df.sample(n=500, random_state=1)
print(len(df_random_sample500))
print("Counts of values of sentiment column in dataframe df_random_sample500: \n", df_random_sample500['sentiment'].value_counts())
value_counts_percent = df_random_sample500['sentiment'].value_counts(normalize=True) * 100
print("Counts of values of sentiment column in dataframe df_random_sample500: \n", value_counts_percent)

df = df_random_sample500

                   id sentiment  \
0  801989080477154944   neutral   
1  801989272341453952  positive   
2  801990978424962944  positive   
3  801996232553963008  positive   
4  801998343442407040   neutral   

                                                text  
0  #ArianaGrande Ari By Ariana Grande 80% Full ht...  
1  Ariana Grande KIIS FM Yours Truly CD listening...  
2  Ariana Grande White House Easter Egg Roll in W...  
3  #CD #Musics Ariana Grande Sweet Like Candy 3.4...  
4  SIDE TO SIDE 😘 @arianagrande #sidetoside #aria...  
Length of dataframe df:  12284
Length of dataframe df:  12284
Length of dataframe df:  12284
Length of dataframe df:  12284
Counts of values of sentiment column in dataframe df: 
 neutral     5937
negative    3972
positive    2375
Name: sentiment, dtype: int64
Counts of values of sentiment column in dataframe df: 
 neutral     48.331162
negative    32.334744
positive    19.334093
Name: sentiment, dtype: float64
12284
500
Counts of values of sentiment colu

In [ ]:
df

,id,sentiment,text
6755,802253826883514368,neutral,Galaxy Note 4 On A Note 7 ROM Gets Samsung's R...
3804,802124452968414976,positive,"The older, the better frase que solo aplica pa..."
8934,799929420853219328,positive,Wow - the world as we know it is slowly turnin...
4207,805686141022334976,neutral,@DAccampoFamily @CLF3rd abortion has nothing t...
11787,805526263418884097,negative,@ali @NolteNC @Reince CA voter registration ac...
...,...,...,...
490,802195201456569984,positive,@verynetwork wowzers!! superb prize!! 😍 #hatch...
946,802207427424845056,positive,Powerful. #MannequinChallenge https://t.co/goY...
7950,802340324840509441,positive,I've been listening to Leonard Cohen nonstop f...
10136,802422711062560768,positive,📷 Playing robots in the Himalayas https://t.co...


# Model
## Load model


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, set_seed


model_name = 'Intel/neural-chat-7b-v3-2'

pipe = pipeline("text-generation", model= model_name, torch_dtype=torch.bfloat16, device_map="auto", add_special_tokens=False)

pipe.tokenizer.pad_token_id = pipe.model.config.eos_token_id

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

# Promting preparation

The script contains functions to construct prompts for the language model. There are different types of prompts:

    Zero-shot prompts
    Few-shot prompts
    One-shot prompts
    Chain of Thought prompts

Each function formats the prompts according to the role (system or user) and content for the language model to generate responses accordingly.

## Structure of prompts

```
{"role": "system",	"content": "Text of System Prompt"},
{"role": "user",	"content": "Question and text to classify"},
{"role": "assistant",	"content": "sentiment"},
{"role": "user",	"content": "Question and another text to classify"}

```



Each function formats the prompts according to the role (system, assistant or user) for the language model to generate responses accordingly.

### Zero-Shot

In [ ]:
def zero_shot_prompt_messages(system_prompt, comment_text, question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": 'Text to classify: ' + str(comment_text) + ' \n' +  str(question)},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

{"role": "system", "content":"Text of System Prompt"},
{"role": "user", "content": "Question and text to classify"},

system_prompt = "Text of System Prompt "
comment_text = "Text to classify"
question = "Question text?"
prompt_messages = zero_shot_prompt_messages(system_prompt, comment_text, question)

print("Example Zero Shot Prompt Messages: ")
print(prompt_messages)


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Example Zero Shot Prompt Messages: 
<s>[INST] <<SYS>>
Text of System Prompt 
<</SYS>>

Text to classify: Text to classify 
Question text? [/INST]


### Few and One-Shot

In [ ]:
def few_shot_prompt_messages(system_prompt, comment_text, question, few_shot_examples):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for user_prompt, assistent_prompt in few_shot_examples:
        messages.append({"role": "user", "content": user_prompt + ' \n' + question})
        messages.append({"role": "assistant", "content": assistent_prompt})

    messages.append({"role": "user", "content":  str(comment_text) + ' \n' + str(question) })

    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt


few_shot_examples = [
    ("Example 01", "neutral"),
    ("Example 02", "positive"),
    ("Example 03", "negative"),
]

one_shot_example = [
    ("Example 01", "neutral"),
]

few_shot_examples_empty = []
question = "Question text "

prompt_messages = few_shot_prompt_messages(system_prompt, comment_text, question, one_shot_example)

print("Example Few Shot Prompt Messages: ")

print(prompt_messages)



Example Few Shot Prompt Messages: 
<s>[INST] <<SYS>>
Text of System Prompt 
<</SYS>>

Example 01 
Question text [/INST] neutral </s><s>[INST] Text to classify 
Question text [/INST]


### Chain of Thoughts

In [ ]:
def cot_prompt_thought_messages(system_prompt, comment_text, question):
      messages = [
        {"role": "system", "content": str(system_prompt)},
        {"role": "user", "content":  str(comment_text) + ' \n' + str(question)}
      ]
      prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
      return prompt

system_prompt = "Text of System Prompt "
comment_text = "Text to classify"
question = "Question text "

prompt_messages = cot_prompt_thought_messages(system_prompt, comment_text, question)


print("Example Chain of Thought Prompt Messages: ")

print(prompt_messages)

Example Chain of Thought Prompt Messages: 
<s>[INST] <<SYS>>
Text of System Prompt 
<</SYS>>

Text to classify 
Question text [/INST]


In [ ]:
def cot_prompt_messages(system_prompt, comment_text, model_thought, question_thought, annotation_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": str(comment_text) + ' \n' + question_thought},
        {"role": "assistant", "content": model_thought},
        {"role": "user", "content": annotation_prompt},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt


system_prompt = "Text of System Prompt"
question_thought = "Think about it and provide some thoughts"
model_thought = "I think the following..."
annotation_prompt = "Now choose one of the following options: Option 1, Option 2, Option 3"
comment_text = "text to classify"

prompt_messages = cot_prompt_messages(system_prompt, comment_text, model_thought, question_thought, annotation_prompt)


print("Example Chain of Thought Prompt Messages: ")

print(prompt_messages)

Example Chain of Thought Prompt Messages: 
<s>[INST] <<SYS>>
Text of System Prompt
<</SYS>>

text to classify 
Think about it and provide some thoughts [/INST] I think the following... </s><s>[INST] Now choose one of the following options: Option 1, Option 2, Option 3 [/INST]


### Chain of Thoughts - Few shot

In [ ]:
from pprint import pprint

def cot_few_shot_THOUGHT_prompt_messages(system_prompt, cot_few_shot_examples, question_thought, comment_text):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for example in cot_few_shot_examples:
        messages.append({"role": "user", "content": example["user"] + question_thought})
        messages.append({"role": "assistant", "content": example["assistant"]})

    messages.append({"role": "user", "content": comment_text + question_thought})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt


system_prompt = "This is the system prompt."
cot_few_shot_examples = [
{"user": "Example Comment 01", "assistant": "Thought 01", "annotation": "Final Annotation 01"},
{"user": "Example Comment 02", "assistant": "Thought 02", "annotation": "Final Annotation 02"},
{"user": "Example Comment 03", "assistant": "Thought 03", "annotation": "Final Annotation 03"},
]
question_thought = " -- Think abput it "
question_annotation = "Now choose one of the following options: Option 1, Option 2, Option 3"
comment_text = "Example Comment 04"

prompt_messages = cot_few_shot_THOUGHT_prompt_messages(system_prompt, cot_few_shot_examples, question_thought, comment_text)

print("Example Chain of Thought Prompt Messages to generate the Thought: ")

pprint(prompt_messages, width=140)

Example Chain of Thought Prompt Messages to generate the Thought: 
('<s>[INST] <<SYS>>\n'
 'This is the system prompt.\n'
 '<</SYS>>\n'
 '\n'
 'Example Comment 01 -- Think abput it [/INST] Thought 01 </s><s>[INST] Example Comment 02 -- Think abput it [/INST] Thought 02 '
 '</s><s>[INST] Example Comment 03 -- Think abput it [/INST] Thought 03 </s><s>[INST] Example Comment 04 -- Think abput it [/INST]')


In [ ]:
def cot_few_shot_prompt_messages(system_prompt, cot_few_shot_examples, model_thought, question_thought, annotation_prompt, comment_text):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for example in cot_few_shot_examples:
        messages.append({"role": "user", "content": example["user"] + question_thought})
        messages.append({"role": "assistant", "content": example["assistant"]})
        messages.append({"role": "user", "content": annotation_prompt})
        messages.append({"role": "assistant", "content": example["annotation"]})

    messages.append({"role": "user", "content": comment_text + question_thought })
    messages.append({"role": "assistant", "content": model_thought })
    messages.append({"role": "user", "content": annotation_prompt })

    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt


system_prompt = "Text of System Prompt"
model_thought = "I think the following..."
question_thought = " -- Think about it and provide some thoughts "
annotation_prompt = "Now choose one of the following options: Option 1, Option 2, Option 3"
comment_text = "text to classify"
cot_few_shot_examples = [
      {"user": "Example Comment 01", "assistant": "Thought 01", "annotation": "Final Annotation 01"},
      {"user": "Example Comment 02", "assistant": "Thought 02", "annotation": "Final Annotation 02"},
      {"user": "Example Comment 03", "assistant": "Thought 03", "annotation": "Final Annotation 03"},
]

prompt_messages = cot_few_shot_prompt_messages(system_prompt, cot_few_shot_examples,  model_thought, question_thought, annotation_prompt, comment_text )


print("Example Chain of Thought Prompt Messages: ")

print(prompt_messages)

Example Chain of Thought Prompt Messages: 
<s>[INST] <<SYS>>
Text of System Prompt
<</SYS>>

Example Comment 01 -- Think about it and provide some thoughts [/INST] Thought 01 </s><s>[INST] Now choose one of the following options: Option 1, Option 2, Option 3 [/INST] Final Annotation 01 </s><s>[INST] Example Comment 02 -- Think about it and provide some thoughts [/INST] Thought 02 </s><s>[INST] Now choose one of the following options: Option 1, Option 2, Option 3 [/INST] Final Annotation 02 </s><s>[INST] Example Comment 03 -- Think about it and provide some thoughts [/INST] Thought 03 </s><s>[INST] Now choose one of the following options: Option 1, Option 2, Option 3 [/INST] Final Annotation 03 </s><s>[INST] text to classify -- Think about it and provide some thoughts [/INST] I think the following... </s><s>[INST] Now choose one of the following options: Option 1, Option 2, Option 3 [/INST]


## Functions

### Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

def calcuate_classification_report(true_labels, predicted_labels):
    # Determine the unique classes and sort them alphabetically
    print(true_labels.value_counts())
    print(predicted_labels.value_counts())
    unique_classes = np.unique(np.concatenate((true_labels.astype(str), predicted_labels.astype(str))))
    unique_classes = unique_classes.astype(str)
    unique_classes.sort()

    classification_report_str = classification_report(true_labels, predicted_labels, labels=unique_classes, zero_division=0)
    classification_report_dict = classification_report(true_labels, predicted_labels, labels=unique_classes, zero_division=0 , output_dict=True)

    return classification_report_str, classification_report_dict

def calculate_kappa(true_labels, predicted_labels):
    # Calculate Cohen's Kappa Score
    kappa = cohen_kappa_score(true_labels, predicted_labels)

    return round(kappa, 4)

def calculate_evaluation_metrics(true_labels, predicted_labels):
    true_labels = true_labels.astype(str)
    predicted_labels = predicted_labels.astype(str)
    print("## Run calcuate_classification_report ")
    classification_report_str, classification_report_dict = calcuate_classification_report(true_labels, predicted_labels)
    print("## Run calculate_kappa ")
    kappa = calculate_kappa(true_labels, predicted_labels)
    return classification_report_str, classification_report_dict, kappa

### Processing

function process_response: processes the language model's responses to extract sentiment keywords.

In [ ]:
def process_response(response, keywords):
    # Convert response to lowercase
    response_lower = response.lower()

    # Search for keywords in response and return the first one found
    for keyword in keywords:
        if keyword.lower() in response_lower:
            return keyword

    return "neutral"
    # If no keyword found, return "neutral"
    # OR CHANGE: If no keyword found, return "notAvailable"
    # return "notAvailable"


# Define the batch processing function
def analyze_text_list(promptsList):
    responses = pipe(promptsList,
                     batch_size=8, # could also be set higher (example to 32); reduce this number if you get an out-of-memory error
                     max_new_tokens=20,
                     do_sample = True,
                     temperature=0.2,
                     top_k=50,
                     top_p=0.95,
                     pad_token_id=pipe.tokenizer.eos_token_id,
                     return_full_text=False)
    categories = [resp[0]["generated_text"] for resp in responses]
    return categories

# Define the batch processing function for the chain of thoughts prompt
def analyze_thought_list(promptsList):
    responses = pipe(promptsList,
                     batch_size=8,  # could also be set higher (example to 32); reduce this number if you get an out-of-memory error
                     max_new_tokens=100,
                     do_sample = True,
                     temperature=0.2,
                     top_k=50,
                     top_p=0.95,
                     pad_token_id=pipe.tokenizer.eos_token_id,
                     return_full_text=False)
    categories = [resp[0]["generated_text"] for resp in responses]
    return categories




In [ ]:
import re
# Function to remove illegal characters
def remove_illegal_characters(text):
    if isinstance(text, str):
        # Remove non-printable characters
        text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    return text

### Function: Run classification

In [ ]:
def run_classification(name, df, output_dir, true_column):
    # Create a copy of the DataFrame to work with inside this function
    df = df.copy()
    print(f"##################\n[START] Promptname: "+ name )
    promtsList = df["prompt"].tolist()

    set_seed(0)

    label_pred = analyze_text_list(promtsList)
    df["response"] = label_pred

    df[true_column] = df[true_column].str.lower()
    keywords = df[true_column].unique()

    new_columname = "pred_" + name
    df[new_columname] = df["response"].apply(lambda x: process_response(x, keywords))
    print(f"##################\n[Count Predictions: ] \n", df[new_columname].value_counts())

    #print(df.head(5))
    print(f"##################\n[Evaluation] Promptname: "+ name )
    classification_report_str, classification_report_dict, kappa = calculate_evaluation_metrics(df[true_column],df[new_columname])
    print(f"\n[Evaluation] Kappa: {round(kappa, 4)}")
    print(f"\n[Evaluation] Classification Report:\n\n", classification_report_str)


    # Use the datestamp and prompt name in the filename
    current_date = datetime.now().strftime('%Y%m%d_%H%M')

    df = df.applymap(remove_illegal_characters)
    df.to_csv(os.path.join(output_dir, f"processed_data_{name}_{df.shape[0]}_rows_{current_date}.csv"), index=False)
    df.to_excel(os.path.join(output_dir, f"processed_data_{name}_{df.shape[0]}_rows_{current_date}.xlsx"))
    print(f"##################\n[Saved] Date and Time : "+ current_date )

    return {
        "prompt": name,
        "df": df,
        "classification_report_str": classification_report_str,
        "classification_report_dict": classification_report_dict,
        "kappa": kappa,
    }


run_thought_classification: Processes prompts to generate thoughts.

In [ ]:
def run_thought_classification(name, df):
    # Create a copy of the DataFrame to work with inside this function
    df = df.copy()
    print(f"##################\n[START] Get thoughts for promptname: "+ name )
    promtsList = df["promptThought"].tolist()

    set_seed(0)

    thought_pred = analyze_thought_list(promtsList)
    df["thought"] = thought_pred

    return df

run_classification_self_consistency: Performs multiple iterations of classification to check the self-consistency of the model's predictions.

In [ ]:
from collections import Counter
def run_classification_self_consistency(name, df, output_dir, true_column, numberOfIterations):
    # Create a copy of the DataFrame to work with inside this function
    df = df.copy()
    df[true_column] = df[true_column].str.lower()
    keywords = df[true_column].unique()

    column_names = []  # List to hold the names of the columns created during iterations


    for iteration in range(numberOfIterations):
        print(f"##################\n[START] Promptname: {name}, Iteration: {iteration + 1}")

        set_seed(iteration)

        promtsList = df["prompt"].tolist()
        label_pred = analyze_text_list(promtsList)
        df["response"] = label_pred

        # Adjusted the column name to include the iteration number
        new_columname = f"pred_{name}_{iteration + 1}"
        column_names.append(new_columname)
        df[new_columname] = df["response"].apply(lambda x: process_response(x, keywords))

        new_columname_response = f"response_{iteration + 1}"
        df[new_columname_response] = df["response"]


    # After all iterations, perform majority vote
    def majority_vote(row):
        # Gather the predictions for this row from all iterations
        predictions = [row[col] for col in column_names]
        # Find the most common prediction and its count
        most_common_prediction, count = Counter(predictions).most_common(1)[0]
        # If the most common prediction is "notAvailable" and there are other labels, find the next most common label
        if most_common_prediction == "notAvailable" and len(set(predictions)) > 1:
            # Get all labels other than "notAvailable"
            other_predictions = [pred for pred in predictions if pred != "notAvailable"]
            # Find the most common among the other labels
            most_common_prediction, _ = Counter(other_predictions).most_common(1)[0]
        return most_common_prediction

    # Create a new column for the majority vote results
    df['majority_vote'] = df.apply(majority_vote, axis=1)



    #print(df.head(5))
    print(f"##################\n[Evaluation] Promptname: "+ name )
    classification_report_str, classification_report_dict, kappa = calculate_evaluation_metrics(df[true_column],df["majority_vote"])
    print(f"\n[Evaluation] Kappa: {round(kappa, 4)}")
    print(f"\n[Evaluation] Classification Report:\n\n", classification_report_str)


    # Use the datestamp and prompt name in the filename
    current_date = datetime.now().strftime('%Y%m%d_%H%M')

    df = df.applymap(remove_illegal_characters)
    df.to_csv(os.path.join(output_dir, f"processed_data_{name}_{df.shape[0]}_rows_{current_date}.csv"), index=False)
    df.to_excel(os.path.join(output_dir, f"processed_data_{name}_{df.shape[0]}_rows_{current_date}.xlsx"))
    print(f"##################\n[Saved] Date and Time : "+ current_date )

    return {
        "prompt": name,
        "df": df,
        "classification_report_str": classification_report_str,
        "classification_report_dict": classification_report_dict,
        "kappa": kappa,
    }


# Run different prompts

In [ ]:
results = {}
true_column = "sentiment"

### Zero-Shot

In [ ]:
# Zero Shot
def run_zeroshot1__classification(df, output_dir):
    prompt_name = "01_zeroshot1"
    system_prompt = "You are an assistant for classifying tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral? Respond only with:  ['positive', 'neutral', 'negative']?"
    df["prompt"] = df["text"].apply(lambda text: zero_shot_prompt_messages(system_prompt, text, question))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)

In [ ]:
run_zeroshot1__classification(df, output_dir)

##################
[START] Promptname: 01_zeroshot1
##################
[Count Predictions: ] 
 neutral     206
negative    158
positive    136
Name: pred_01_zeroshot1, dtype: int64
##################
[Evaluation] Promptname: 01_zeroshot1
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
neutral     206
negative    158
positive    136
Name: pred_01_zeroshot1, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5534

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.78      0.73      0.75       171
     neutral       0.72      0.65      0.68       227
    positive       0.61      0.81      0.70       102

    accuracy                           0.71       500
   macro avg       0.70      0.73      0.71       500
weighted avg       0.72      0.71      0.71       500

##################
[Saved] Date and Time : 20231211_1633


In [ ]:
# Zero Shot
def run_zeroshot2__classification(df, output_dir):
    prompt_name = "02_zeroshot2"
    system_prompt = "You are a reliable chat assistant for classifying tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral? Respond with: /n 'positive' /n 'neutral' /n 'negative' /n"
    df["prompt"] = df["text"].apply(lambda text: zero_shot_prompt_messages(system_prompt, text, question))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)


run_zeroshot2__classification(df, output_dir)

##################
[START] Promptname: 02_zeroshot2
##################
[Count Predictions: ] 
 neutral     304
negative    140
positive     56
Name: pred_02_zeroshot2, dtype: int64
##################
[Evaluation] Promptname: 02_zeroshot2
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
neutral     304
negative    140
positive     56
Name: pred_02_zeroshot2, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5243

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.86      0.70      0.77       171
     neutral       0.64      0.85      0.73       227
    positive       0.75      0.41      0.53       102

    accuracy                           0.71       500
   macro avg       0.75      0.66      0.68       500
weighted avg       0.74      0.71      0.70       500

##################
[Saved] Date and Time : 20231211_1634


In [ ]:
# Zero Shot
def run_zeroshot3__classification(df, output_dir):
    prompt_name = "03_zeroshot3"
    system_prompt = "Please help me to classify tweets for their sentiment."
    question = " --- Question:  Is the sentiment for this tweet positive, negative or neutral (Respond only with:  ['positive', 'neutral', 'negative'])?"
    df["prompt"] = df["text"].apply(lambda text: zero_shot_prompt_messages(system_prompt, text, question))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)

run_zeroshot3__classification(df, output_dir)

##################
[START] Promptname: 03_zeroshot3
##################
[Count Predictions: ] 
 neutral     215
negative    159
positive    126
Name: pred_03_zeroshot3, dtype: int64
##################
[Evaluation] Promptname: 03_zeroshot3
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
neutral     215
negative    159
positive    126
Name: pred_03_zeroshot3, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5532

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.77      0.72      0.75       171
     neutral       0.71      0.67      0.69       227
    positive       0.64      0.79      0.71       102

    accuracy                           0.71       500
   macro avg       0.71      0.73      0.71       500
weighted avg       0.72      0.71      0.71       500

##################
[Saved] Date and Time : 20231211_1634


In [ ]:
# Zero Shot

def run_zeroshot4__classification(df, output_dir):
    prompt_name = "04_zeroshot4"
    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "Question: Is the sentiment for this tweet positive, negative or neutral. Do not reply using a complete sentence, respond with only one word: ['positive', 'neutral', 'negative']"
    df["prompt"] = df["text"].apply(lambda text: zero_shot_prompt_messages(system_prompt, text, question))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)


run_zeroshot4__classification(df, output_dir)

##################
[START] Promptname: 04_zeroshot4
##################
[Count Predictions: ] 
 neutral     405
negative     94
positive      1
Name: pred_04_zeroshot4, dtype: int64
##################
[Evaluation] Promptname: 04_zeroshot4
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
neutral     405
negative     94
positive      1
Name: pred_04_zeroshot4, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.2706

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.83      0.46      0.59       171
     neutral       0.53      0.94      0.68       227
    positive       1.00      0.01      0.02       102

    accuracy                           0.59       500
   macro avg       0.79      0.47      0.43       500
weighted avg       0.73      0.59      0.51       500

##################
[Saved] Date and Time : 20231211_1634


### One-Shot

In [ ]:
# One Shot
def run_One_shot1__classification(df, output_dir):
    prompt_name = "05_One_shot1"
    # Define the question and context
    example_p = [
    (" Want to start reading Paper Towns tomorrow. My sister told me the movie was awesome! ", "positive"),
    ]

    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"


    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)

run_One_shot1__classification(df, output_dir)

##################
[START] Promptname: 05_One_shot1
##################
[Count Predictions: ] 
 negative    237
positive    132
neutral     131
Name: pred_05_One_shot1, dtype: int64
##################
[Evaluation] Promptname: 05_One_shot1
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    237
positive    132
neutral     131
Name: pred_05_One_shot1, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5038

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.65      0.90      0.75       171
     neutral       0.77      0.44      0.56       227
    positive       0.61      0.78      0.68       102

    accuracy                           0.67       500
   macro avg       0.68      0.71      0.67       500
weighted avg       0.70      0.67      0.65       500

##################
[Saved] Date and Time : 20231211_1635


In [ ]:
# One Shot
def run_One_shot2__classification(df, output_dir):
    prompt_name = "06_One_shot2"
    # Define the question and context
    example_p = [
        ("Missing  Foo Fighters concert on the 12th at the gorge though so that's a major bummer",  "negative"),
    ]
    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"

    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)

run_One_shot2__classification(df, output_dir)

##################
[START] Promptname: 06_One_shot2
##################
[Count Predictions: ] 
 negative    205
positive    185
neutral     110
Name: pred_06_One_shot2, dtype: int64
##################
[Evaluation] Promptname: 06_One_shot2
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    205
positive    185
neutral     110
Name: pred_06_One_shot2, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.4652

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.68      0.82      0.74       171
     neutral       0.76      0.37      0.50       227
    positive       0.50      0.91      0.65       102

    accuracy                           0.63       500
   macro avg       0.65      0.70      0.63       500
weighted avg       0.68      0.63      0.61       500

##################
[Saved] Date and Time : 20231211_1636


In [ ]:
def run_One_shot3__classification(df, output_dir):
    prompt_name = "07_One_shot3"


    example_p = [
    ("Is anyone going to see Shawn in  NYC tomorrow", "neutral"),
    ]
    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"

    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))
    results[prompt_name] = run_classification(prompt_name, df, output_dir, true_column)

run_One_shot3__classification(df, output_dir)


##################
[START] Promptname: 07_One_shot3
##################
[Count Predictions: ] 
 negative    247
positive    189
neutral      64
Name: pred_07_One_shot3, dtype: int64
##################
[Evaluation] Promptname: 07_One_shot3
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    247
positive    189
neutral      64
Name: pred_07_One_shot3, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.428

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.63      0.91      0.75       171
     neutral       0.81      0.23      0.36       227
    positive       0.49      0.91      0.64       102

    accuracy                           0.60       500
   macro avg       0.65      0.68      0.58       500
weighted avg       0.69      0.60      0.55       500

##################
[Saved] Date and Time : 20231211_1637


## FewShot

In [ ]:
# Few Shot Classification
def run_Few_shot1__classification(df, output_dir):
    prompt_name = "08_Few_shot1"

    # Examples
    example_p = [
    ("Want to start reading Paper Towns tomorrow. My sister told me the movie was awesome! ", "positive"),
    ("Missing  Foo Fighters concert on the 12th at the gorge though so that's a major bummer",  "negative"),
    ("Is anyone going to see Shawn in  NYC tomorrow", "neutral"),
    ]

    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"


    # Applying the prompt to the DataFrame
    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))
    results[prompt_name] = run_classification(prompt_name, df, output_dir, true_column)

run_Few_shot1__classification(df, output_dir)


##################
[START] Promptname: 08_Few_shot1
##################
[Count Predictions: ] 
 negative    214
positive    155
neutral     131
Name: pred_08_Few_shot1, dtype: int64
##################
[Evaluation] Promptname: 08_Few_shot1
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    214
positive    155
neutral     131
Name: pred_08_Few_shot1, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.4758

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.65      0.82      0.73       171
     neutral       0.75      0.43      0.55       227
    positive       0.55      0.84      0.67       102

    accuracy                           0.65       500
   macro avg       0.65      0.70      0.65       500
weighted avg       0.68      0.65      0.63       500

##################
[Saved] Date and Time : 20231211_1638


In [ ]:
# Few Shot
def run_Few_shot2__classification(df, output_dir):
    prompt_name = "09_Few_shot2"
    # # Examples
    example_p = [
        ("Just finished a great workout session, feeling energized and ready for the day!", "positive"),
        ("Can't believe my vacation is over. Back to the daily grind tomorrow.", "negative"),
        ("Thinking about what to cook for dinner tonight. Maybe pasta?", "neutral"),
        ("Absolutely loved the new art exhibit at the museum. So inspiring!", "positive"),
        ("Feeling under the weather today, hope it's not the flu.", "negative"),
        ("Anyone know a good place for coffee in downtown?", "neutral")
    ]


    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"

    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)

run_Few_shot2__classification(df, output_dir)

##################
[START] Promptname: 09_Few_shot2
##################
[Count Predictions: ] 
 neutral     220
negative    194
positive     86
Name: pred_09_Few_shot2, dtype: int64
##################
[Evaluation] Promptname: 09_Few_shot2
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
neutral     220
negative    194
positive     86
Name: pred_09_Few_shot2, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5004

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.67      0.76      0.71       171
     neutral       0.68      0.66      0.67       227
    positive       0.72      0.61      0.66       102

    accuracy                           0.68       500
   macro avg       0.69      0.68      0.68       500
weighted avg       0.69      0.68      0.68       500

##################
[Saved] Date and Time : 20231211_1640


In [ ]:
# Few Shot
def run_Few_shot3__classification(df, output_dir):
    prompt_name = "10_Few_shot3"
    # Define the question and context
    example_p = [
    ("Really enjoyed the new coffee shop on Main Street, the ambiance was perfect!", "positive"),
    ("Feeling frustrated with the constant roadworks on my daily commute.", "negative"),
    ("Looks like it's going to be another ordinary day at the office.", "neutral")]


    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"




    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))
    results[prompt_name]  = run_classification(prompt_name, df, output_dir, true_column)

run_Few_shot3__classification(df, output_dir)

##################
[START] Promptname: 10_Few_shot3
##################
[Count Predictions: ] 
 negative    193
neutral     170
positive    137
Name: pred_10_Few_shot3, dtype: int64
##################
[Evaluation] Promptname: 10_Few_shot3
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    193
neutral     170
positive    137
Name: pred_10_Few_shot3, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5287

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.69      0.78      0.74       171
     neutral       0.75      0.56      0.64       227
    positive       0.61      0.82      0.70       102

    accuracy                           0.69       500
   macro avg       0.68      0.72      0.69       500
weighted avg       0.70      0.69      0.69       500

##################
[Saved] Date and Time : 20231211_1641


## Chain of Thought Prompting


In [ ]:
# Chain of Thought Prompting
def run_COT_classification(df, output_dir):
    prompt_name = "11_COT"

    # System prompt and questions
    system_p = "Please help me to classify tweets for their sentiment."
    question_thought = "--- Is the sentiment for this tweet positive, negative or neutral. Think about it and provide some thoughts:"
    question_p = "--- Now choose one of the following options: Is the sentiment for the tweet positive, negative or neutral take your thoughts into account and respond with only one word 'positive', 'neutral' or 'negative'. One-word reply:"

    # Applying the thought process prompt to the DataFrame
    df["promptThought"] = df["text"].apply(lambda text: cot_prompt_thought_messages(system_p, text, question_thought))
    df = run_thought_classification(prompt_name, df)

    # Applying the final prompt, which includes the thought process, to the DataFrame
    df["prompt"] = df.apply(lambda row: cot_prompt_messages(system_p, row["text"], row["thought"], question_thought, question_p), axis=1)
    results[prompt_name] = run_classification(prompt_name, df, output_dir, true_column)

run_COT_classification(df, output_dir)


##################
[START] Get thoughts for promptname: 11_COT


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 11_COT


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[Count Predictions: ] 
 negative    185
positive    183
neutral     132
Name: pred_11_COT, dtype: int64
##################
[Evaluation] Promptname: 11_COT
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    185
positive    183
neutral     132
Name: pred_11_COT, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.4698

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.72      0.78      0.75       171
     neutral       0.74      0.43      0.55       227
    positive       0.48      0.86      0.62       102

    accuracy                           0.64       500
   macro avg       0.65      0.69      0.64       500
weighted avg       0.68      0.64      0.63       500

##################
[Saved] Date and Time : 20231211_1645


In [ ]:
def run_COT_fewshot_classification(df, output_dir):
    prompt_name = "12_COT"

    # System prompt and questions
    system_p = "Please help me to classify tweets for their sentiment."
    question_thought = "--- Is the sentiment for this tweet positive, negative or neutral. Think about it and provide some thoughts:"
    question_p = "--- Now choose one of the following options: 'positive','neutral' or 'negative'. One-word reply:"


    # Define few-shot examples for Chain of Thought
    cot_few_shot_examples = [
        {
            "user": "I had a fantastic day at the beach with my family!",
            "assistant": "This tweet expresses happiness and enjoyment, indicating a positive sentiment about spending time at the beach.",
            "annotation": "positive"
        },
        {
            "user": "Feeling down today. Sometimes things just don't go as planned.",
            "assistant": "The tweet mentions feelings of being 'down' and disappointment. This suggests a negative sentiment due to things not going as planned.",
            "annotation": "negative"
        },
        {
            "user": "Just another regular day at work. Nothing special.",
            "assistant": "The tweet describes a routine day without any specific positive or negative emotions. This seems like a neutral sentiment.",
            "annotation": "neutral"
        },
        {
            "user": "Can't believe how bad the traffic was today. Totally ruined my mood.",
            "assistant": "The tweet talks about frustration with bad traffic and a ruined mood, which clearly indicates a negative sentiment.",
            "annotation": "negative"
        }
    ]

    # Applying the COT with few-shot examples to the DataFrame
    df["promptThought"] = df["text"].apply(lambda text: cot_few_shot_THOUGHT_prompt_messages(system_p, cot_few_shot_examples, question_thought, text))
    df = run_thought_classification(prompt_name, df)

    # Applying the final prompt, which includes the thought process, to the DataFrame
    df["prompt"] = df.apply(lambda row: cot_few_shot_prompt_messages(system_p, cot_few_shot_examples, row["thought"], question_thought, question_p, row["text"]), axis=1)
    results[prompt_name] = run_classification(prompt_name, df, output_dir, true_column)


run_COT_fewshot_classification(df, output_dir)




##################
[START] Get thoughts for promptname: 12_COT


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 12_COT


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[Count Predictions: ] 
 positive    297
neutral     109
negative     94
Name: pred_12_COT, dtype: int64
##################
[Evaluation] Promptname: 12_COT
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
positive    297
neutral     109
negative     94
Name: pred_12_COT, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.0161

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.33      0.18      0.23       171
     neutral       0.54      0.26      0.35       227
    positive       0.20      0.57      0.29       102

    accuracy                           0.30       500
   macro avg       0.36      0.34      0.29       500
weighted avg       0.40      0.30      0.30       500

##################
[Saved] Date and Time : 20231211_1651


## Self-Consistency Prompting


In [ ]:
# self_consistency
def run_self_consistency1__classification(df, output_dir):
    prompt_name = "13_self_consistency1"
    system_p = "You are an assistant for classifiying tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral? Only respond with ['positive', 'neutral', 'negative']:"

    numberOfIterations = 5

    df["prompt"] = df["text"].apply(lambda text: zero_shot_prompt_messages(system_p, text, question))
    results[prompt_name]  = run_classification_self_consistency(prompt_name, df, output_dir, true_column, numberOfIterations)

run_self_consistency1__classification(df, output_dir)


##################
[START] Promptname: 13_self_consistency1, Iteration: 1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 13_self_consistency1, Iteration: 2


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 13_self_consistency1, Iteration: 3


...

##################
[START] Promptname: 13_self_consistency1, Iteration: 4


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 13_self_consistency1, Iteration: 5


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[Evaluation] Promptname: 13_self_consistency1
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    193
positive    162
neutral     145
Name: majority_vote, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.5076

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.72      0.81      0.76       171
     neutral       0.75      0.48      0.59       227
    positive       0.54      0.86      0.67       102

    accuracy                           0.67       500
   macro avg       0.67      0.72      0.67       500
weighted avg       0.70      0.67      0.66       500

##################
[Saved] Date and Time : 20231211_1652


In [ ]:
# self_consistency
def run_self_consistency2__classification(df, output_dir):
    prompt_name = "14_self_consistency2"

    example_p = [
        ("Missing Foo Fighters concert on the 12th at the gorge though so that's a major bummer",  "negative"),
    ]
    system_p = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"



    numberOfIterations = 5
    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_p, text, question, example_p))

    results[prompt_name]  = run_classification_self_consistency(prompt_name, df, output_dir, true_column, numberOfIterations)

run_self_consistency2__classification(df, output_dir)


##################
[START] Promptname: 14_self_consistency2, Iteration: 1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 14_self_consistency2, Iteration: 2


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 14_self_consistency2, Iteration: 3


...

##################
[START] Promptname: 14_self_consistency2, Iteration: 4


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 14_self_consistency2, Iteration: 5


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[Evaluation] Promptname: 14_self_consistency2
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    198
positive    174
neutral     128
Name: majority_vote, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.4774

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.68      0.79      0.73       171
     neutral       0.74      0.42      0.54       227
    positive       0.53      0.91      0.67       102

    accuracy                           0.65       500
   macro avg       0.65      0.71      0.65       500
weighted avg       0.68      0.65      0.63       500

##################
[Saved] Date and Time : 20231211_1657


In [ ]:
# self_consistency
def run_self_consistency3__classification(df, output_dir):
    prompt_name = "15_self_consistency3"

    example_p = [
      ("Want to start reading Paper Towns tomorrow. My sister told me the movie was awesome! ", "positive"),
      ("Missing  Foo Fighters concert on the 12th at the gorge though so that's a major bummer",  "negative"),
      ("Is anyone going to see Shawn in  NYC tomorrow", "neutral"),
    ]

    system_prompt = "Please help me to classify tweets for their sentiment."
    question = "--- Is the sentiment for this tweet positive, negative or neutral (only respond with ['positive', 'neutral', 'negative'])?"



    numberOfIterations = 5
    df["prompt"] = df["text"].apply(lambda text: few_shot_prompt_messages(system_prompt, text, question, example_p))

    results[prompt_name]  = run_classification_self_consistency(prompt_name, df, output_dir, true_column, numberOfIterations)

run_self_consistency3__classification(df, output_dir)


##################
[START] Promptname: 15_self_consistency3, Iteration: 1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 15_self_consistency3, Iteration: 2


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 15_self_consistency3, Iteration: 3


...

##################
[START] Promptname: 15_self_consistency3, Iteration: 4


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[START] Promptname: 15_self_consistency3, Iteration: 5


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


##################
[Evaluation] Promptname: 15_self_consistency3
## Run calcuate_classification_report 
neutral     227
negative    171
positive    102
Name: sentiment, dtype: int64
negative    216
positive    152
neutral     132
Name: majority_vote, dtype: int64
## Run calculate_kappa 

[Evaluation] Kappa: 0.4779

[Evaluation] Classification Report:

               precision    recall  f1-score   support

    negative       0.65      0.82      0.72       171
     neutral       0.74      0.43      0.55       227
    positive       0.57      0.85      0.69       102

    accuracy                           0.65       500
   macro avg       0.65      0.70      0.65       500
weighted avg       0.68      0.65      0.64       500

##################
[Saved] Date and Time : 20231211_1702


# Save

In [ ]:
import pickle

current_date = datetime.now().strftime('%Y%m%d_%H%M')

with open(os.path.join(output_dir, f"results_{current_date}.pkl"), 'wb') as fp:
    pickle.dump(results, fp)


# Plot

In [ ]:
for key in results.keys():
    print(key)

01_zeroshot1
02_zeroshot2
03_zeroshot3
04_zeroshot4
05_One_shot1
06_One_shot2
07_One_shot3
08_Few_shot1
09_Few_shot2
10_Few_shot3
11_COT
12_COT
13_self_consistency1
14_self_consistency2
15_self_consistency3


In [ ]:
import matplotlib.pyplot as plt

current_date = datetime.now().strftime('%Y%m%d_%H%M')

# Initialize lists to store the metrics and labels for x-axis
kappa_values = []
weighted_f1_scores = []
macro_avg_f1_scores = []
accuracy_values = []
keys = []

# Loop over the results dictionary to extract metrics
for key, value in results.items():
    if 'kappa' in value and 'classification_report_dict' in value:
        keys.append(key)
        kappa_values.append(value['kappa'])
        weighted_f1_scores.append(value['classification_report_dict']['weighted avg']['f1-score'])
        macro_avg_f1_scores.append(value['classification_report_dict']['macro avg']['f1-score'])
        accuracy_values.append(value['classification_report_dict']['accuracy'])

# Convert keys into positions on the x-axis
x_positions = range(len(keys))

# Plotting
plt.figure(figsize=(10, 5))  # Set the figure size as needed

# Creating a scatter plot for each metric using different markers and line styles
plt.scatter(x_positions, kappa_values, c='black', marker='o', label='Kappa', s=50, facecolors='none')  # Circle
plt.scatter(x_positions, weighted_f1_scores, c='black', marker='x', label='Weighted F1-Score', s=50)  # X marker
plt.scatter(x_positions, macro_avg_f1_scores, c='black', marker='+', label='Macro Avg F1-Score', s=50)  # Plus marker
plt.scatter(x_positions, accuracy_values, marker='^', label='Accuracy', s=50, facecolors='none', edgecolors='black')  # Triangle marker

# Adding labels and title
plt.xlabel('Keys')
plt.ylabel('Scores')
plt.title('Comparison of Kappa, Weighted F1-Score, and Macro Avg F1-Score')

# Adding a legend to differentiate the points with markers
plt.legend()

# Set the position of the x ticks
plt.xticks(x_positions, keys, rotation=90)

# Show the plot
plt.tight_layout()

# Save the figure
plt.savefig(os.path.join(output_dir, f"results_{current_date}_metrics_comparison_plot.pdf"), format='pdf', bbox_inches='tight')
plt.savefig(os.path.join(output_dir, f"results_{current_date}_metrics_comparison_plot.png"), format='png', bbox_inches='tight')
plt.savefig(os.path.join(output_dir, f"results_{current_date}_metrics_comparison_plot.eps"), format='eps', bbox_inches='tight')

plt.show()



# Table

In [ ]:
import pandas as pd

# Create a DataFrame from the metrics
df_metrics = pd.DataFrame({
    'Key': keys,
    'Kappa': kappa_values,
    'Weighted F1-Score': weighted_f1_scores,
    'Macro Avg F1-Score': macro_avg_f1_scores,
    'Accuracy': accuracy_values
})

# Display the DataFrame
print(df_metrics)

# Save the DataFrame to a CSV file
df_metrics.to_csv(os.path.join(output_dir, f"results_{current_date}_metrics_table.csv"), index=False)

# If you want to save it to Excel, use:
df_metrics.to_excel(os.path.join(output_dir, f"results_{current_date}_metrics_table.xlsx"), index=False)


In [ ]:
# Sort the DataFrame by the 'Kappa' column in descending order
df_metrics_sorted = df_metrics.sort_values('Kappa', ascending=False)

# Display the sorted DataFrame
print(df_metrics_sorted)

# Terminate

In [ ]:
from google.colab import runtime
runtime.unassign()